In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from datasetV2 import *
import cv2
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from generate_results import *
from model3 import jacard_coef,jacard_coef_loss,DiceLoss

In [ ]:
!pwd

In [2]:
gpu_available = tf.test.is_gpu_available()
print(gpu_available)
tf.config.list_physical_devices('GPU')

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


2022-06-11 10:17:29.456712: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-11 10:17:29.510927: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-11 10:17:29.600839: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-11 10:17:29.601201: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# model = load_model('../Tensorflow_remote/model_parcial16V3_originalSize.h5',custom_objects={'jacard_coef_loss' : jacard_coef_loss,'jacard_coef' : jacard_coef})
# model = load_model('../Tensorflow_remote/model_parcial16V3_originalSize.h5')

model = load_model('trained_models/model_reduced_jac_jac_sizeX2_10ep_lr5E5_batch16.h5',custom_objects={'jacard_coef_loss' : jacard_coef_loss,'jacard_coef' : jacard_coef})
# model = load_model('trained_models/model_reduced_jac_jac_sizeX2_1ep_lr1E4_batch8.h5',custom_objects={'jacard_coef' : jacard_coef})

# model = load_model('trained_models/model_reduced_jac_jac_dice_sizeX2_20ep_lr5E5_var_batch16_new_jac.h5',custom_objects={'jacard_coef_loss' : jacard_coef_loss,'jacard_coef' : jacard_coef, 'DiceLoss' : DiceLoss})


IMAGE_HEIGHT = 224
IMAGE_WIDTH = 144


IMAGE_HEIGHT = 448
IMAGE_WIDTH = 288



In [ ]:
test_path =  'original csvs/test_dataset.csv' 

BATCH_SIZE = 16 
BUFFER_SIZE = 1000

test_images, test_masks = load_data(test_path)
test_dataset = tf_dataset(test_images, test_masks,batch_size=BATCH_SIZE, buffer_size=BUFFER_SIZE,shuffle=False)




In [ ]:
original = []
masks = []
predicted = []
ind = 0
for elem in tqdm(test_dataset):
    break
    predictions = model.predict(elem[0])
    for item in predictions:
        predicted.append(item > 0.5)

    for item in elem[0]: # image
        original.append(item)

    for item in elem[1]: # masks 
        masks.append(item)
    ind = ind + 1
    if ind == 32:
        break
        

In [ ]:
print(original[0].shape)
print(masks[0].shape)
print(predicted[0].shape)
print(np.unique(predicted[0]))

print(len(original))
print(len(masks))
print(len(predicted))

index = 147
plt.subplot(1, 3, 1)
plt.imshow(original[index])
plt.subplot(1, 3, 2)
plt.imshow(masks[index])
plt.subplot(1, 3, 3)
plt.imshow(predicted[index])

In [ ]:
# generate_results_reduced(dirname='test_results/model_reduced_jac_jac_dice_sizeX2_20ep_lr5E5_var_batch16_new_jac/',predictions= predicted,masks= masks,images_path= test_images)

generate_results_reduced(dirname='test_results/model_reduced_jac_jac_dice_sizeX2_20ep_lr5E5_var_batch16_new_jac/',model= model,dataset= test_dataset)

In [ ]:
plt.imshow(cv2.imread(test_images[1]))

In [ ]:
results = model.evaluate(test_dataset,batch_size=16)

In [ ]:
predict = model.predict(test_dataset.take(1))

In [ ]:
plt.imshow(predict[0])


In [ ]:
index = 308
pred = predicted[index].astype(np.uint8)
mask = masks[index].numpy().astype(np.uint8)

mask = cv2.resize(mask,(720,1160),interpolation = cv2.INTER_NEAREST)
pred = cv2.resize(pred,(720,1160),interpolation = cv2.INTER_NEAREST)

img = cv2.imread(test_images[index])
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

res_pred = img
# pred[pred == 1] = 255

pred = pred.astype(np.uint8)

contours,_ = cv2.findContours(pred, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)

# rect = cv2.minAreaRect(contours[0])
# print(rect)
# box = cv2.boxPoints(rect)
# box = box.astype(int)

heigth = 70
width = 180

pt2 = np.float32([[0,0], [width,0], [0,heigth], [width,heigth]])

approx = cv2.approxPolyDP(contours[0], 0.035 * cv2.arcLength(contours[0], False), True)

box = np.squeeze(approx, axis=1)
print(box)
# if box[0][0] < box[2][0]:
#     print('if')
#     pt12 = np.float32([box[0],box[1],box[3],box[2]])
#     box_aux = [box[0],box[1],box[3],box[2]]
#     pt12 = np.float32([box[0],box[3],box[1],box[2]])

# else:
#     pt12 = np.float32([box[1],box[2],box[0],box[3]])
#     box_aux = [box[1],box[2],box[0],box[3]]
#     pt12 = np.float32([box[3],box[2],box[0],box[1]])
#     pt12 = np.float32([box[1],box[0],box[2],box[3]])
if box[0][0] < box[2][0]:
    print('if')
    pt12 = np.float32([box[0],box[3],box[1],box[2]])
    if box[0][1] > box[2][1]:
        pt12 = np.float32([box[3],box[2],box[0],box[1]])
else: 
    print('else')
    pt12 = np.float32([box[1],box[0],box[2],box[3]])
    print(pt12)
    if box[0][1] > box[2][1]:
        pt12 = np.float32([box[2],box[1],box[3],box[0]])



# pt12 = np.float32([box[0],box[3],box[1],box[2]])
# pt12 = np.float32([box[3],box[2],box[0],box[1]])
# pt12 = np.float32([box[2],box[1],box[3],box[0]])


f = cv2.getPerspectiveTransform(pt12,pt2)
f2 = cv2.warpPerspective(img,f,(width,heigth))


ff = np.squeeze(approx, axis=1)
# print(ff)
# box = [i for in ]
# print(box)
# print(box_aux)
box = ff
# box = box_aux
# print(approx[:4])
    # draws boundary of contours.
# for i in a
# cv2.drawContours(pred, [approx[:4]], 0, (255), 5)
for i in range(0,4):
    cv2.circle(pred,(box[i][0], box[i][1]), 20, (i+1)*50, -1)

plt.imshow(pred,cmap='gray')
plt.imshow(f2)

In [ ]:
import shutil

batch_size = 8
# index = 0

dirname = 'test_results/model_fullNet/results/'
listdir = os.listdir(dirname)
for dir in listdir:
    shutil.rmtree(dirname + dir)
# reescale bin image
os.mkdir(dirname + '!Todas')
num = 0
for index in tqdm(range(0,len(predicted))):
    pred = predicted[index].astype(np.uint8)
    mask = masks[index].numpy().astype(np.uint8)

    mask = cv2.resize(mask,(720,1160),interpolation = cv2.INTER_NEAREST)
    pred = cv2.resize(pred,(720,1160),interpolation = cv2.INTER_NEAREST)

    img = cv2.imread(test_images[index])
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.imshow(img)

    res_pred = img
    pred[pred == 1] = 255
    # print(np.unique(pred))



    pred = pred.astype(np.uint8)

    contours,_ = cv2.findContours(pred, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    # if len(contours)  == 0:
    #     break
    approx = cv2.approxPolyDP(contours[0], 0.05 * cv2.arcLength(contours[0], True), True)

    # rect = cv2.minAreaRect(contours[0])
    # box = cv2.boxPoints(rect)
    # print(box)
    # print(approx)
    box = np.squeeze(approx, axis=1)
    # cv2.drawContours(mask, [approx[:4]], 0, (255), 5)
    # for i in range(0,4):
    #     cv2.circle(mask,(box[i][0], box[i][1]), 25, (i+1)*50, -1)

    heigth = 70
    width = 180

    pt2 = np.float32([[0,0], [width,0], [0,heigth], [width,heigth]])
    # print(box)
    if box[0][0] < box[2][0]:
        pt12 = np.float32([box[0],box[3],box[1],box[2]])
    else:
        pt12 = np.float32([box[1],box[0],box[2],box[3]])
        

    f = cv2.getPerspectiveTransform(pt12,pt2)
    f2 = cv2.warpPerspective(img,f,(width,heigth))


    gray = cv2.cvtColor(f2,cv2.COLOR_BGR2GRAY)
    # gray = f2[:,:,2]
    # gray = cv2.bilateralFilter(gray,11,17,17)
    canny = cv2.Canny(gray,40,200)
    # lines = cv2.HoughLinesP(canny,1,np.pi/180,10, maxLineGap=150)

    lines = cv2.HoughLinesP(canny,0.001,np.pi/180,3, minLineLength=40, maxLineGap=180)

    # if lines is None:
    #     break
    

    hough = np.zeros((heigth,width), np.uint8)
    hough = canny.copy()


    # for rho,theta in lines[0]:
    #     a = np.cos(theta)
    #     b = np.sin(theta)
    #     x0 = a*rho
    #     y0 = b*rho
    #     x1 = int(x0 + 1000*(-b))
    #     y1 = int(y0 + 1000*(a))
    #     x2 = int(x0 - 1000*(-b))
    #     y2 = int(y0 - 1000*(a))

    #     cv2.line(hough,(x1,y1),(x2,y2),(255),1)
    


    if lines is not None:
        for line in lines:
            x1,y1,x2,y2 = line[0]
            cv2.line(hough,(x1,y1),(x2,y2),(255),1)

    kernel = np.ones((int(heigth * 0.1)), dtype=np.uint8) * 255
    # plt.imshow(gray)

    opening = cv2.morphologyEx(hough, cv2.MORPH_OPEN, kernel)
    # plt.imshow(opening)
    boxop = getBoundingBoxFromMask(opening).astype(int)
    
    result = f2[boxop[0]:boxop[2],boxop[1]:boxop[3]]

    if not result.any():
        result = f2.copy()
    resultSeg = result.copy()

    he, wi,dm = resultSeg.shape
    div = int(wi/7)

    divs = [i * div + 5 for i in range(1,7)]
    divs[0] = divs[0] - 5
    divs[1] = divs[1] - 5
    divs[-2] = divs[-2] - 5
    divs[-1] = divs[-1] - 5

    # print(divs,width,resultSeg.shape)


    divImgList = [];

    for i in divs:
        # divImgList.append(resultSeg)
        resultSeg[:,i-1:i+1] = 0
        
    # plt.imshow(canny)
    dir = dirname + str(index) + '_0' +  str(index2) + '/'
    # num = ((index + 1) * (index2 + 1))
    num = num + 1
    if num < 10:
        dir = dirname + '0' + str( num ) + '/'
    else:
        dir = dirname + str( num ) + '/'

    # dir = 'result/' + str(index) + '_0' +  str(index2) + '_'
    os.mkdir(dir)

    # ret, threshed_img = cv2.threshold(cv2.cvtColor(result,cv2.COLOR_BGR2GRAY),100, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY)
    # threshed_img = cv2.adaptiveThreshold(cv2.cvtColor(result,cv2.COLOR_BGR2GRAY), 255, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY, 11 , 10)
    # ret, threshed_img = cv2.threshold(result[:,:,0],100, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY)

    # cv2.imwrite(dir + '/'+ lista_dir[index][index2],cv2.cvtColor(img,cv2.COLOR_RGB2BGR))
    cv2.imwrite(dir + '1_original.jpg',cv2.cvtColor(img,cv2.COLOR_RGB2BGR))
    cv2.imwrite(dir + '3_prediction.jpg',pred)
    cv2.imwrite(dir + '2_mask.jpg',mask * 255)
    cv2.imwrite(dir + '4_perspectiva.jpg',cv2.cvtColor(f2,cv2.COLOR_RGB2BGR))
    cv2.imwrite(dir + '5_canny.jpg',canny)
    cv2.imwrite(dir + '5.6_morphology.jpg',opening)
    if len(result) > 0:
        cv2.imwrite(dir + '6_recortadaCanny.jpg',cv2.cvtColor(result,cv2.COLOR_RGB2BGR))
        cv2.imwrite(dir + '7_recortadaCanny_CaracteresSegmentados.jpg',cv2.cvtColor(resultSeg,cv2.COLOR_RGB2BGR))
        cv2.imwrite(dirname + '!Todas/' + str(num) + '_7_recortadaCanny_CaracteresSegmentados.jpg',cv2.cvtColor(resultSeg,cv2.COLOR_RGB2BGR))
    # cv2.imwrite(dirname + '!Todas/' + str(index) + '_0' +  str(index2) + '_8_recortadaCanny_CaracteresSegmentados.jpg',cv2.cvtColor(threshed_img,cv2.COLOR_RGB2BGR))


In [ ]:
def getBoundingBoxFromMask(Y):
    """Convert mask Y to a bounding box, assumes 0 as background nonzero object"""
    cols, rows = np.nonzero(Y)
    if len(cols)==0: 
        return np.zeros(4, dtype=np.float32)
    top_row = np.min(rows)
    left_col = np.min(cols)
    bottom_row = np.max(rows)
    right_col = np.max(cols)
    return np.array([left_col, top_row, right_col, bottom_row], dtype=np.float32)

In [ ]:
path = '../ccpd_base/'
listdir = os.listdir(path)

plt.imshow(cv2.imread(train_images[0]))

In [ ]:
plt.imshow(mask - pred)

In [ ]:
print(type(train_dataset.take(1)))

In [ ]:
# train_dataset.shuffle(false)

for elem in train_dataset.take(2):
    plt.subplot(1, 3, 1)
    plt.imshow(elem[0][0])
    plt.subplot(1, 3, 2)
    plt.imshow(elem[1][0])

    print(elem[0].shape)
    pred = model.predict(elem[0])
    print('pred ',pred.shape)
    plt.subplot(1, 3, 3)
    plt.imshow(pred[0])

